# imports

In [2]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from z3 import *
from xgboost import XGBClassifier
from pmlb import fetch_data

set_option(rational_to_decimal=True)

In [3]:
import multiprocessing
print("Número de núcleos disponíveis:", multiprocessing.cpu_count())
set_param("parallel.enable", True)
set_param("threads", 12)

Número de núcleos disponíveis: 12


# model

In [4]:
from z3 import *
import numpy as np


class XGBoostExplainer:
    def __init__(self, model, data):
        self.model = model
        self.data = data.values
        self.columns = data.columns
        self.max_categories = 2

        set_option(rational_to_decimal=True)
        self.categoric_features = self.get_categoric_features(self.data)
        self.T_model = self.model_trees_expression(self.model)
        self.T = self.T_model

    def explain(self, instance, reorder="asc"):
        self.I = self.instance_expression(instance)
        self.D, self.D_add = self.decision_function_expression(self.model, [instance])
        return self.explain_expression(self.I, And(self.T, self.D_add), self.D, self.model, reorder)

    def explain_prob(self, instance, reorder="asc", threshold_margin=0, target_threshold=None):
        self.I = self.instance_expression(instance)
        self.D, self.D_add = self.decision_function_expression(self.model, [instance])
        return self.explain_expression_prob(self.I, And(self.T, self.D_add), self.D, self.model, reorder, threshold_margin, target_threshold)

    def get_categoric_features(self, data: np.ndarray):
        categoric_features = []
        for i in range(data.shape[1]):
            feature_values = data[:, i]
            unique_values = np.unique(feature_values)
            if len(unique_values) <= self.max_categories:
                categoric_features.append(self.columns[i])

        return categoric_features

    def feature_constraints(self, constraints=[]):
        """TODO
        esperado receber limites das features pelo usuário
        formato previso: matriz/dataframe [feaature, min/max, valor]
        constraaint_expression = "constraaint_df_to_feature()"
        """
        return

    def model_trees_expression(self, model):
        df = model.get_booster().trees_to_dataframe()
        if model.get_booster().feature_names == None:
            feature_map = {f"f{i}": col for i, col in enumerate(self.columns)}
            df["Feature"] = df["Feature"].replace(feature_map)

        df["Split"] = df["Split"].round(4)
        self.booster_df = df
        class_index = 0  # if model.n_classes_ == 2:
        all_tree_formulas = []

        for tree_index in df["Tree"].unique():
            tree_df = df[df["Tree"] == tree_index]
            o = Real(f"o_{tree_index}_{class_index}")

            if len(tree_df) == 1 and tree_df.iloc[0]["Feature"] == "Leaf":
                leaf_value = tree_df.iloc[0]["Gain"]
                all_tree_formulas.append(And(o == leaf_value))
                continue
            path_formulas = []

            def get_conditions(node_id):
                conditions = []
                current_node = tree_df[tree_df["ID"] == node_id]
                if current_node.empty:
                    return conditions

                parent_node = tree_df[
                    (tree_df["Yes"] == node_id) | (tree_df["No"] == node_id)
                ]
                if not parent_node.empty:
                    parent_data = parent_node.iloc[0]
                    feature = parent_data["Feature"]
                    split_value = parent_data["Split"]
                    x = Real(feature)
                    if parent_data["Yes"] == node_id:
                        conditions.append(x < split_value)
                    else:
                        conditions.append(x >= split_value)
                    conditions = get_conditions(parent_data["ID"]) + conditions

                return conditions

            for _, node in tree_df[tree_df["Feature"] == "Leaf"].iterrows():
                leaf_value = node["Gain"]
                leaf_id = node["ID"]
                conditions = get_conditions(leaf_id)
                path_formula = And(*conditions)
                implication = Implies(path_formula, o == leaf_value)
                path_formulas.append(implication)

            all_tree_formulas.append(And(*path_formulas))
        return And(*all_tree_formulas)

    def get_init_value(self, model, x, estimator_variables):
        estimator_pred = Solver()
        estimator_pred.add(self.I)
        estimator_pred.add(self.T)
        if estimator_pred.check() == sat:
            solvermodel = estimator_pred.model()
            total_sum = sum(
                float(solvermodel.eval(var).as_fraction()) for var in estimator_variables
            )
        else:
            total_sum = 0
            print("estimator error")
        self.predicted_margin = model.predict(x, output_margin=True)[0]
        init_value = self.predicted_margin - total_sum
        self.init_value = init_value
        return init_value

    def decision_function_expression(self, model, x):
        n_classes = 1 if model.n_classes_ <= 2 else model.n_classes_
        predicted_class = model.predict(x)[0]
        self.predicted_class = predicted_class
        n_estimators = int(len(model.get_booster().get_dump()) / n_classes)
        estimator_variables = [Real(f"o_{i}_0") for i in range(n_estimators)] # _0 only for binary classification
        self.estimator_variables = estimator_variables
        init_value = self.get_init_value(model, x, estimator_variables)
        # print("init:", round(init_value, 2))

        equation_list = []

        estimator_sum = Real("estimator_sum")
        equation_o = estimator_sum == Sum(estimator_variables)
        equation_list.append(equation_o)

        decision = Real("decision")
        equation_list.append(decision == estimator_sum + init_value)

        if predicted_class == 0:
            final_equation = decision < 0
        else:
            final_equation = decision > 0

        return final_equation, And(equation_list)

    def instance_expression(self, instance):
        formula = [Real(self.columns[i]) == value for i, value in enumerate(instance)]
        return formula

    def explain_expression(self, I, T_s, D_s, model, reorder):
        i_expression = I.copy()

        importances = model.feature_importances_
        non_zero_indices = np.where(importances != 0)[0]

        if reorder == "asc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]
        elif reorder == "desc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(-importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]

        for feature in i_expression.copy():
            # print("\n---removed", feature)
            i_expression.remove(feature)

            # prove(Implies(And(And(i_expression), T), D))
            if self.is_proved(Implies(And(And(i_expression), T_s), D_s)):
                continue
                # print('proved')
            else:
                # print('not proved')
                i_expression.append(feature)
        # print(self.is_proved(Implies(And(And(i_expression), T_s), D_s)))
        return i_expression

    def explain_expression_prob(self, I, T_s, D_s, model, reorder, threshold_margin, target_threshold):
        i_expression = I.copy()

        importances = model.feature_importances_
        non_zero_indices = np.where(importances != 0)[0]

        if reorder == "asc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]
        elif reorder == "desc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(-importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]

        threshold = 0

        if target_threshold:
            threshold = target_threshold
        elif threshold_margin != 0:
            threshold = self.predicted_margin * threshold_margin/100
            # print("margin:", self.predicted_margin, "accepted margin:", threshold)
        self.xai_predicted_margin = self.predicted_margin

        for feature in i_expression.copy():
            # print("\n---removed", feature)
            i_expression.remove(feature)

            if self.is_proved_sat(And(And(i_expression), T_s), threshold):
                # print('proved')
                continue
            else:
                # print('not proved -- added back')
                i_expression.append(feature)
        return i_expression


    def is_proved(self, decision_exp):
        s = Solver()
        s.add(Not(decision_exp))
        if s.check() == unsat:
            return True
        else:
            # print(s.model())
            return False

    def is_proved_sat(self, decision_exp, threshold):
      decision = Real("decision")

      debug = Real("debug") == 0
      predicted_class = self.predicted_class

      if predicted_class == 0:
        estmax = Optimize()
        estmax.add(decision_exp)
        estmax.add(debug)
        maxvalue = estmax.maximize(decision)
        if estmax.check() == sat:
            # print("\nmax sat", maxvalue.value())
            try:
              if float(maxvalue.value().as_fraction()) > threshold:
                  return False # can change class
              else:
                  self.xai_predicted_margin = float(maxvalue.value().as_fraction())
            except:
              print("error max =", maxvalue.value())
              return False
        else:
            print("error")

      if predicted_class == 1:
        estmin = Optimize()
        estmin.add(decision_exp)
        estmin.add(debug)
        minvalue = estmin.minimize(decision)
        if estmin.check() == sat:
            # print("\nmin sat", minvalue.value())
            try:
              if float(minvalue.value().as_fraction()) < threshold:
                  return False # can change class
              else:
                  self.xai_predicted_margin = float(minvalue.value().as_fraction())
            except:
              print("error min =", minvalue.value())
              return False
        else:
            print("error")

      if predicted_class == 0:
        self.solvermodel = estmax.model()
      if predicted_class == 1:
        self.solvermodel = estmin.model()
      return True

# tests

## iris

In [5]:
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
y[y == 2] = 0  # converte em binario

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=101
)

model = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

explainer = XGBoostExplainer(model, X_train)

In [6]:
print(len(X_test))
print(model.score(X_test, y_test))
y_pred

30
0.9333333333333333


array([0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0])

In [7]:
X_test[y_pred == 0].index

Index([33, 16, 43, 123, 146, 1, 147, 32, 31, 122, 127, 42, 48, 114, 25, 41], dtype='int64')

In [8]:
results = []
for i in X_test[y_pred == 0].index:
    sample = X_test.loc[i].values
    xai = explainer.explain_prob(sample, reorder="asc")
    xaiprob_initial = explainer.xai_predicted_margin
    len_xai_initial = len(xai)

    xai = explainer.explain_prob(sample, reorder="asc", target_threshold=xaiprob_initial - 0.01)
    xaiprob_final = explainer.xai_predicted_margin
    len_xai_final = len(xai)

    results.append({
        "index": i,
        "class": 0,
        "xaiprob_initial": xaiprob_initial,
        "len_xai_initial": len_xai_initial,
        "xaiprob_final": xaiprob_final,
        "len_xai_final": len_xai_final
    })
df_results = pd.DataFrame(results)
df_results

,index,class,xaiprob_initial,len_xai_initial,xaiprob_final,len_xai_final
0,33,0,-0.973060,2,-1.143638,3
1,16,0,-0.973056,2,-1.143635,3
2,43,0,-0.973056,2,-1.143635,3
3,123,0,-0.531414,2,-1.155039,2
4,146,0,-2.040002,2,-4.083848,3
5,1,0,-0.973056,2,-1.143635,3
6,147,0,-3.975426,2,-4.865418,3
7,32,0,-0.973056,2,-1.143635,3
8,31,0,-0.973056,2,-1.143635,3
9,122,0,-3.975426,2,-5.215529,3


In [9]:
results = []
for i in X_test[y_pred == 1].index:
    sample = X_test.loc[i].values
    xai = explainer.explain_prob(sample, reorder="asc")
    xaiprob_initial = explainer.xai_predicted_margin
    len_xai_initial = len(xai)

    xai = explainer.explain_prob(sample, reorder="asc", target_threshold=xaiprob_initial + 0.01)
    xaiprob_final = explainer.xai_predicted_margin
    len_xai_final = len(xai)

    results.append({
        "index": i,
        "class": 0,
        "xaiprob_initial": xaiprob_initial,
        "len_xai_initial": len_xai_initial,
        "xaiprob_final": xaiprob_final,
        "len_xai_final": len_xai_final
    })
df_results = pd.DataFrame(results)
df_results

,index,class,xaiprob_initial,len_xai_initial,xaiprob_final,len_xai_final
0,129,0,0.722122,2,1.883706,3
1,50,0,1.369809,2,3.355599,3
2,68,0,1.467798,2,2.541687,3
3,53,0,1.727301,2,2.820987,3
4,74,0,1.727303,2,1.897882,3
5,88,0,1.727299,2,3.069216,3
6,96,0,1.727303,2,1.897882,3
7,134,0,0.151577,4,0.151577,4
8,80,0,1.727301,2,2.820987,3
9,90,0,1.727303,2,2.971770,3


## adult dataset

In [35]:
adult = fetch_data('adult')
adult.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39.0,7,77516.0,9,13.0,4,1,1,4,1,2174.0,0.0,40.0,39,1
1,50.0,6,83311.0,9,13.0,2,4,0,4,1,0.0,0.0,13.0,39,1
2,38.0,4,215646.0,11,9.0,0,6,1,4,1,0.0,0.0,40.0,39,1
3,53.0,4,234721.0,1,7.0,2,6,0,2,1,0.0,0.0,40.0,39,1
4,28.0,4,338409.0,9,13.0,2,10,5,2,0,0.0,0.0,40.0,5,1


In [36]:
X = adult.drop('target', axis=1)
y = adult['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=101
)

model = XGBClassifier(n_estimators=30, max_depth=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

explainer = XGBoostExplainer(model, X_train)

In [37]:
print(len(X_test))
print(model.score(X_test, y_test))
y_pred

9769
0.8659023441498618


array([1, 1, 1, ..., 1, 1, 1])

In [47]:
sample = X_test[X_test.index == 17510]
print(explainer.explain_prob(sample.values[0]))
print(explainer.xai_predicted_margin)

[capital-loss == 0, capital-gain == 0, education-num == 13, relationship == 3]
0.08755603616


In [48]:
print(model.predict(sample))

[1]


In [ ]:
# results = []
# for i in X_test[y_pred == 0].index:
#     sample = X_test.loc[i].values
#     xai = explainer.explain_prob(sample, reorder="asc")
#     xaiprob_initial = explainer.xai_predicted_margin
#     len_xai_initial = len(xai)

#     xai = explainer.explain_prob(sample, reorder="asc", target_threshold=xaiprob_initial - 0.01)
#     xaiprob_final = explainer.xai_predicted_margin
#     len_xai_final = len(xai)

#     results.append({
#         "index": i,
#         "class": 0,
#         "xaiprob_initial": xaiprob_initial,
#         "len_xai_initial": len_xai_initial,
#         "xaiprob_final": xaiprob_final,
#         "len_xai_final": len_xai_final
#     })
# df_results = pd.DataFrame(results)
# df_results

,index,class,xaiprob_initial,len_xai_initial,xaiprob_final,len_xai_final
0,25242,0,-0.044041,14,-0.044041,14
1,18463,0,-0.107054,9,-0.235938,9
2,8541,0,-0.492402,10,-0.752300,11
3,33498,0,-0.019865,13,-0.019865,14
4,34701,0,-0.197970,14,-0.197970,14
5,32519,0,-0.306206,13,-0.445489,12
6,34570,0,-0.195087,11,-0.429066,11
7,7263,0,-0.145633,11,-0.715617,12
8,3466,0,-0.375374,10,-0.386114,11
9,13535,0,-0.223543,9,-0.601157,9


In [ ]:
# results = []
# for i in X_test[y_pred == 1].index:
#     sample = X_test.loc[i].values
#     xai = explainer.explain_prob(sample, reorder="asc")
#     xaiprob_initial = explainer.xai_predicted_margin
#     len_xai_initial = len(xai)

#     xai = explainer.explain_prob(sample, reorder="asc", target_threshold=xaiprob_initial + 0.01)
#     xaiprob_final = explainer.xai_predicted_margin
#     len_xai_final = len(xai)

#     results.append({
#         "index": i,
#         "class": 0,
#         "xaiprob_initial": xaiprob_initial,
#         "len_xai_initial": len_xai_initial,
#         "xaiprob_final": xaiprob_final,
#         "len_xai_final": len_xai_final
#     })
# df_results = pd.DataFrame(results)
# df_results

,index,class,xaiprob_initial,len_xai_initial,xaiprob_final,len_xai_final
0,30658,0,0.317430,9,1.069127,9
1,25493,0,1.002439,9,1.220849,10
2,11136,0,0.034109,11,0.233938,11
3,3814,0,0.072905,10,0.639263,11
4,743,0,0.166484,10,0.863320,10
5,47228,0,0.394212,7,0.736613,8
6,21872,0,0.482364,9,0.496912,9
7,16644,0,0.327519,9,0.893877,10
8,34372,0,0.029761,11,0.409006,11
9,13979,0,0.105647,8,0.526180,8


## mushroom

In [15]:
mushroom = fetch_data("mushroom")
mushroom.head()

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,target
0,2,0,7,0,6,1,0,0,8,1,...,3,5,0,2,1,4,1,4,6,0
1,0,3,9,0,0,1,0,0,4,0,...,7,7,0,2,1,4,0,2,0,0
2,2,3,8,1,6,1,1,0,0,1,...,7,7,0,2,1,0,0,3,0,0
3,2,2,3,1,4,1,0,0,4,0,...,0,5,0,2,1,2,3,4,3,1
4,3,0,0,0,6,1,0,0,8,1,...,7,5,0,2,1,4,0,5,6,0


In [16]:
X = mushroom.drop('target', axis=1)
y = mushroom['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=101
)

model = XGBClassifier(n_estimators=30, max_depth=3, learning_rate=1)
model.fit(X_train, y_train)

explainer = XGBoostExplainer(model, X_train)

In [17]:
print(X.shape)
print(model.score(X_test, y_test))
model.predict(X_test[0:30])

(8124, 22)
1.0


array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1])

In [19]:
# sample = X_test.values[0]
# results, xai_probs, xai_lengths_base = describe_output_margin_results(X, model, explainer, sample, threshold_margin=30, n_generated_samples=10000)
# print(xai_lengths_base)
# print(xai_probs)
# results

In [21]:
# sample = X_test.values[1]
# results, xai_probs, xai_lengths_base = describe_output_margin_results(X, model, explainer, sample, threshold_margin=30, n_generated_samples=10000)
# print(xai_lengths_base)
# print(xai_probs)
# results